In [1]:
import pandas as pd
import numpy as np
from collections import Counter


# Create the MLS schedule from game data

Data available from https://www.kaggle.com/datasets/josephvm/major-league-soccer-dataset?select=matches.csv


Unfortunately the dataset is not complete. The following games are missing in dataset and have to be added:

2006: "Chivas FC" vs "Colorado Rapids"

2005: "Sporting Kansas City" vs "LA Galaxy"

2004: "DC United" vs "New England Revolution", "Columbus Crew SC" vs "New York Red Bulls", "San Jose Earthquakes" vs "Sporting Kansas City", "FC Dalls" vs "LA Galaxy"




In [2]:
df = pd.read_csv("../../raw_data/MLS/MLS_game_data_kaggle.csv")
df = df.drop_duplicates("id")


/var/folders/vx/56f7nbnd4x11hf9xxfw22kg40000gn/T/ipykernel_56359/2356028469.py:1: DtypeWarning: Columns (11,14,50,102,123,125,129,131,133,135,137,139,141,143,145,147,149,157,158,160,161,163,165,167,169,171,173,184,186,187,205,206) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../raw_data/MLS/MLS_game_data_kaggle.csv")


In [3]:
# Only look at regular season games
df = df[df["part_of_competition"].str.contains("Regular")]
df = df[df["game_status"] != "Abandoned"]
# Exclude 2022 season because it is still running at this point (2023-01-05)
# Exclude 2020 season because of covid
df = df[df["year"] != 2022]
df = df[df["year"] != 2020]
df = df[["home", "away", "year"]]
df = df.rename(columns={"home": "Home", "away": "Away", "year": "Year"})


In [4]:
def add_game_to_df(df, home, away, year):
    tmp_series = pd.Series({"Home": home, "Away": away, "Year": year})
    return pd.concat([df, tmp_series.to_frame().T], ignore_index=True)


In [5]:
grpy = df.groupby("Year")
df_list = {}
# iterate ofer pandas groupby object and save schedule for each season
for year, season_df in grpy:
    if year == 2004:
        season_df = add_game_to_df(
            season_df, "DC United", "New England Revolution", year
        )
        season_df = add_game_to_df(
            season_df, "Columbus Crew SC", "New York Red Bulls", year
        )
        season_df = add_game_to_df(
            season_df, "San Jose Earthquakes", "Sporting Kansas City", year
        )
        season_df = add_game_to_df(season_df, "FC Dallas", "LA Galaxy", year)
    if year == 2005:
        season_df = add_game_to_df(season_df, "Sporting Kansas City", "LA Galaxy", year)
    if year == 2006:
        season_df = add_game_to_df(season_df, "Chivas USA", "Colorado Rapids", year)
    season_df.to_csv(f"../../prepared_data/schedules/MLS/{year}.csv", index=False)

